In [3]:
!pip uninstall -y tensorflow tensorflow-intel tensorflow-gpu

Found existing installation: tensorflow 2.13.1
Uninstalling tensorflow-2.13.1:
  Successfully uninstalled tensorflow-2.13.1
Found existing installation: tensorflow-intel 2.13.1
Uninstalling tensorflow-intel-2.13.1:
  Successfully uninstalled tensorflow-intel-2.13.1


You can safely remove it manually.


In [5]:
!pip uninstall -y tensorflow-io-gcs-filesystem 0.31.0
!pip uninstall -y tensorflow-estimator         2.13.0

Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
  Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0


Found existing installation: tensorflow-estimator 2.13.0
Uninstalling tensorflow-estimator-2.13.0:
  Successfully uninstalled tensorflow-estimator-2.13.0


In [5]:
!pip list | findstr tensorflow

In [7]:
!pip cache purge

Files removed: 244


In [6]:
#!pip install --upgrade pip

In [6]:
!pip install tensorflow==2.15.0 --upgrade

  Using cached tensorflow-2.15.0-cp39-cp39-win_amd64.whl.metadata (3.6 kB)
   ---------------------------------------- 0.0/300.8 MB ? eta -:--:--
   - -------------------------------------- 12.1/300.8 MB 63.1 MB/s eta 0:00:05
   -- ------------------------------------- 21.2/300.8 MB 53.8 MB/s eta 0:00:06
   ---- ----------------------------------- 34.9/300.8 MB 58.4 MB/s eta 0:00:05
   ----- ---------------------------------- 43.0/300.8 MB 53.6 MB/s eta 0:00:05
   ------- -------------------------------- 57.7/300.8 MB 57.4 MB/s eta 0:00:05
   ------- -------------------------------- 59.8/300.8 MB 54.4 MB/s eta 0:00:05
   --------- ------------------------------ 73.9/300.8 MB 52.4 MB/s eta 0:00:05
   ----------- ---------------------------- 83.9/300.8 MB 52.0 MB/s eta 0:00:05
   ------------- ------------------------- 100.4/300.8 MB 55.3 MB/s eta 0:00:04
   ------------- ------------------------- 103.8/300.8 MB 53.9 MB/s eta 0:00:04
   --------------- ----------------------- 120.8/300.8

In [7]:
!pip list | findstr tensorflow

tensorflow                   2.15.0
tensorflow-estimator         2.15.0
tensorflow-intel             2.15.0
tensorflow-io-gcs-filesystem 0.31.0


In [9]:
nvcc --version

NameError: name 'nvcc' is not defined

In [8]:
# 📚 IMPORTAR LIBRERÍAS PRINCIPALES
import tensorflow as tf
import os

# ⚙️ CONFIGURAR USO DE GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar solo la primera GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("✅ GPU configurada correctamente.")
    except RuntimeError as e:
        print(e)
else:
    print("❌ No se encontró GPU. Usando CPU.")



❌ No se encontró GPU. Usando CPU.


In [18]:
# 📚 Librerías Necesarias
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import joblib

# ✅ Parámetros Globales
IMG_SIZE = (128, 128)  # Tamaño de las imágenes
BATCH_SIZE = 32        # Tamaño del batch
EPOCHS = 50            # Número de épocas
COLOR_MODE = 'grayscale'  # Imágenes en escala de grises
AUTOTUNE = tf.data.AUTOTUNE

In [19]:
# ✅ Rutas de Datos
ruta_train_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ Cargar Datos
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

# ✅ Preprocesamiento de Imágenes
def cargar_datos(df, ruta_img, es_entrenamiento=True):
    ids = df['id'].values
    labels = df['clip_count'].values if 'clip_count' in df.columns else None
    
    def preprocesar_imagen(id, label=None):
        img_path = tf.strings.join([ruta_img, "/clips-", tf.strings.as_string(id), ".png"])
        img = tf.io.read_file(img_path)
        img = tf.image.decode_png(img, channels=1)
        img = tf.image.resize(img, IMG_SIZE)
        img = img / 255.0
        return (img, tf.cast(label, tf.float32)) if label is not None else img
    
    if es_entrenamiento:
        dataset = tf.data.Dataset.from_tensor_slices((ids, labels))
        dataset = dataset.map(lambda id, label: preprocesar_imagen(id, label), num_parallel_calls=AUTOTUNE)
    else:
        dataset = tf.data.Dataset.from_tensor_slices(ids)
        dataset = dataset.map(lambda id: preprocesar_imagen(id), num_parallel_calls=AUTOTUNE)
    
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

# ✅ Crear Datasets
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=42)
dataset_train = cargar_datos(df_train, ruta_train_img, es_entrenamiento=True)
dataset_val = cargar_datos(df_val, ruta_train_img, es_entrenamiento=True)
dataset_test = cargar_datos(df_test, ruta_test_img, es_entrenamiento=False)

In [20]:
# ✅ Definir el Modelo CNN
def crear_modelo():
    modelo = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='linear')  # Salida de regresión
    ])
    
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error',
        metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')]
    )
    return modelo

# ✅ Crear Modelo
modelo = crear_modelo()
modelo.summary()

c:\Users\mikel\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,516,609 (24.86 MB)

 Trainable params: 6,516,161 (24.86 MB)

 Non-trainable params: 448 (1.75 KB)

In [21]:
# ✅ Callbacks
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5)

# ✅ Entrenar Modelo
historial = modelo.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr]
)

# ✅ Graficar Métricas
plt.plot(historial.history['rmse'], label='RMSE (Train)')
plt.plot(historial.history['val_rmse'], label='RMSE (Validation)')
plt.legend()
plt.show()

Epoch 1/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 184s 483ms/step - loss: 601.7779 - rmse: 22.1491 - val_loss: 1074.2441 - val_rmse: 32.7757 - learning_rate: 0.0010
Epoch 2/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 175s 466ms/step - loss: 149.5768 - rmse: 12.2265 - val_loss: 141.8510 - val_rmse: 11.9101 - learning_rate: 0.0010
Epoch 3/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 186s 495ms/step - loss: 144.1265 - rmse: 12.0037 - val_loss: 153.9276 - val_rmse: 12.4068 - learning_rate: 0.0010
Epoch 4/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 184s 491ms/step - loss: 141.2737 - rmse: 11.8824 - val_loss: 370.2022 - val_rmse: 19.2406 - learning_rate: 0.0010
Epoch 5/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 177s 471ms/step - loss: 130.5571 - rmse: 11.4241 - val_loss: 1871.3793 - val_rmse: 43.2594 - learning_rate: 0.0010
Epoch 6/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 184s 490ms/step - loss: 129.6989 - rmse: 11.3877 - val_loss: 58.5122 - val_rmse: 7.6493 - learning_rate: 0.0010
Epoch 7/50
375/375 ━━━━━━━━━━━━━━━━━━━━ 188s 501ms/step - loss: 132.6027 - r

KeyboardInterrupt: 

In [ ]:
# ✅ Evaluación
train_loss, train_rmse = modelo.evaluate(dataset_train)
val_loss, val_rmse = modelo.evaluate(dataset_val)
print(f"🔹 RMSE (Train): {train_rmse:.2f}")
print(f"🔹 RMSE (Validation): {val_rmse:.2f}")

# ✅ Predicciones
predicciones = modelo.predict(dataset_test).flatten()
df_test['clip_count'] = np.round(predicciones).astype(int)

# ✅ Guardar Resultados
df_test[['id', 'clip_count']].to_csv('submission.csv', index=False)
print("✅ Archivo 'submission.csv' generado correctamente.")